# Market Learning
*Leveraging machine learning to help you learn the market and invest your money*

### Objective
The Market Learning project is intended to provide an adaptive, fact-based introduction of the stock market to a user with limited experience looking for help with personal investment decisions.

### Outline
- Data Collection
- Data Cleaning
- Visualization
- Prediction
- Conclusion

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import yfinance as yf

### Data Collection
Get a list of tickers for various securities
Get yfinance data for those tickers

In [3]:
# Get a list of tickers for different types of securities: stocks, ETFs, mutual funds, etc.
page_stocks = requests.get('https://stockanalysis.com/stocks/')
page_stocks.status_code

200

In [6]:
# Initialize lists of stock ticker info
ticker = []
name = []
market_cap = []
stock_price = []
day_change = []
stock_volume = []
market_time = []
security_type = []
primary_exchange = []
industry = []
dividend_yield = []
PE_ratio = []
other_categories = []


In [8]:
# webscrape the stockanalysis page for the following css selector: '.svelte-1jtwn20 a'
soup = bs(page_stocks.content, 'html.parser')
results = soup.select('.svelte-1jtwn20 a')
for item in results:
    ticker.append(item.get_text())
    name.append(item.get('title'))
    security_type.append('stock')
len(ticker), ticker

(1000,
 ['A',
  'AA',
  'AAC',
  'AACG',
  'AACI',
  'AADI',
  'AAIC',
  'AAL',
  'AAMC',
  'AAME',
  'AAN',
  'AAOI',
  'AAON',
  'AAP',
  'AAPL',
  'AAT',
  'AAU',
  'AB',
  'ABB',
  'ABBV',
  'ABC',
  'ABCB',
  'ABCL',
  'ABCM',
  'ABEO',
  'ABEV',
  'ABG',
  'ABIO',
  'ABM',
  'ABNB',
  'ABOS',
  'ABR',
  'ABSI',
  'ABST',
  'ABT',
  'ABUS',
  'ABVC',
  'AC',
  'ACA',
  'ACAB',
  'ACAC',
  'ACAD',
  'ACAH',
  'ACAQ',
  'ACAX',
  'ACB',
  'ACBA',
  'ACCD',
  'ACCO',
  'ACDC',
  'ACEL',
  'ACER',
  'ACET',
  'ACGL',
  'ACGN',
  'ACHC',
  'ACHL',
  'ACHR',
  'ACHV',
  'ACI',
  'ACIU',
  'ACIW',
  'ACLS',
  'ACLX',
  'ACM',
  'ACMR',
  'ACN',
  'ACNB',
  'ACNT',
  'ACON',
  'ACOR',
  'ACR',
  'ACRE',
  'ACRO',
  'ACRS',
  'ACRV',
  'ACRX',
  'ACST',
  'ACT',
  'ACTG',
  'ACU',
  'ACVA',
  'ACXP',
  'ADAG',
  'ADAL',
  'ADAP',
  'ADBE',
  'ADC',
  'ADCT',
  'ADD',
  'ADEA',
  'ADER',
  'ADES',
  'ADEX',
  'ADI',
  'ADIL',
  'ADM',
  'ADMA',
  'ADMP',
  'ADN',
  'ADNT',
  'ADOC',
  'ADP'

In [5]:
goog = yf.Ticker('GOOG')
goog.info


{'address1': '1600 Amphitheatre Parkway',
 'city': 'Mountain View',
 'state': 'CA',
 'zip': '94043',
 'country': 'United States',
 'phone': '650 253 0000',
 'website': 'https://www.abc.xyz',
 'industry': 'Internet Content & Information',
 'sector': 'Communication Services',
 'longBusinessSummary': 'Alphabet Inc. offers various products and platforms in the United States, Europe, the Middle East, Africa, the Asia-Pacific, Canada, and Latin America. It operates through Google Services, Google Cloud, and Other Bets segments. The Google Services segment provides products and services, including ads, Android, Chrome, hardware, Gmail, Google Drive, Google Maps, Google Photos, Google Play, Search, and YouTube. It is also involved in the sale of apps and in-app purchases and digital content in the Google Play store; and Fitbit wearable devices, Google Nest home products, Pixel phones, and other devices, as well as in the provision of YouTube non-advertising services. The Google Cloud segment o

In [6]:
goog.history()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-03-15 00:00:00-04:00,93.540001,97.250000,93.040001,96.550003,38367300,0.0,0.0
2023-03-16 00:00:00-04:00,96.570000,101.970001,95.870003,101.070000,54499500,0.0,0.0
2023-03-17 00:00:00-04:00,100.839996,103.489998,100.750000,102.459999,76140300,0.0,0.0
2023-03-20 00:00:00-04:00,101.059998,102.580002,100.790001,101.930000,26033900,0.0,0.0
2023-03-21 00:00:00-04:00,101.980003,105.959999,101.860001,105.839996,33122800,0.0,0.0
2023-03-22 00:00:00-04:00,105.139999,107.510002,104.209999,104.220001,32336900,0.0,0.0
2023-03-23 00:00:00-04:00,105.889999,107.100998,105.410004,106.260002,31385800,0.0,0.0
2023-03-24 00:00:00-04:00,105.739998,106.160004,104.739998,106.059998,25245000,0.0,0.0
2023-03-27 00:00:00-04:00,105.320000,105.400002,102.629997,103.059998,25393400,0.0,0.0
